In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.model_selection as ms

# random forest package
from sklearn.ensemble import RandomForestRegressor

# xgBoost package
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder


# print("xgboost", xgboost.__version__)

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'


In [2]:
# load data 
df = pd.read_csv("data.csv",encoding="latin-1")